### start

In [1]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import pickle
import cloudpickle
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import warnings
import boto3
import matplotlib as plt

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
# ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)

# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')

import sagemaker_wallets.wallet_modeler as wm
import sagemaker_wallets.wallet_preprocessor as wp
import sagemaker_wallets.workflow_orchestrator as wo
import sage_utils.config_validation as ucv
import utils as u
from utils import ConfigError

# reload all modules
modules = [
    wm, wp, wo,
    u, ucv
]

from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#b45827"


# load all configs
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


u.export_code(
    code_directories=[
        'sagemaker_wallets',
        # 'sage_utils',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
    output_file="temp/sagemaker_code.py"
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

[15/Jul/25 23:11:46] INFO [credentials.load:1352] Found credentials in shared credentials file: ~/.aws/credentials


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/jeremymeadow/Library/Application Support/sagemaker/config.yaml


[15/Jul/25 23:11:46] INFO [utils.export_code:1667] Consolidation complete. All files are saved in temp/sagemaker_code.py
[15/Jul/25 23:11:46] MILESTONE [2567990582.<module>:115] Good morning, let's get to work


# Initial ETLs

# Code begins

### Initiate orchestrator

In [2]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config)

### Load, preprocess, and upload data

#### load data

In [3]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffixes = [
    '231107',
    '240306'
]
workflow_orch.load_training_data(date_suffixes)

[15/Jul/25 23:11:47] MILESTONE [workflow_orchestrator.load_training_data:109] <PROD> Loading training data for 2 periods: ['231107', '240306']
[15/Jul/25 23:11:48] INFO [workflow_orchestrator.load_training_data:127] Training data loaded successfully: 8 splits, 398,964 total rows


##### inspect features

In [ ]:
# # Create combined NaN count and describe statistics
# nan_counts = workflow_orch.training_data['x_train'].isna().sum()
# describe_stats = workflow_orch.training_data['x_train'].describe().T

# # Combine into single DataFrame
# combined_stats = pd.concat([
#     nan_counts.rename('nan_count'),
#     describe_stats
# ], axis=1).sort_index()

# u.display_full(combined_stats.sort_index())

#### preprocess data

In [4]:
preprocessor = wp.SageWalletsPreprocessor(sage_wallets_config)
preprocessed_data = preprocessor.preprocess_training_data(workflow_orch.training_data)

[15/Jul/25 23:11:52] INFO [wallet_preprocessor.preprocess_training_data:51] Starting preprocessing for SageMaker XGBoost compatibility...
[15/Jul/25 23:11:52] INFO [wallet_preprocessor._handle_missing_values:186] Filled NaN values in 100 columns for x_train.
[15/Jul/25 23:11:52] INFO [wallet_preprocessor._combine_x_y_data:283] Merged y df with target var cw_crypto_net_gain/crypto_inflows/winsorized with X data.
[15/Jul/25 23:11:52] INFO [wallet_preprocessor.preprocess_training_data:86] Preprocessed train: 87,727 rows × 222 cols.
[15/Jul/25 23:11:52] INFO [wallet_preprocessor._handle_missing_values:186] Filled NaN values in 100 columns for x_test.
[15/Jul/25 23:11:52] INFO [wallet_preprocessor._combine_x_y_data:283] Merged y df with target var cw_crypto_net_gain/crypto_inflows/winsorized with X data.
[15/Jul/25 23:11:52] INFO [wallet_preprocessor.preprocess_training_data:86] Preprocessed test: 16,450 rows × 222 cols.
[15/Jul/25 23:11:52] INFO [wallet_preprocessor._handle_missing_values:

#### upload data

In [6]:
# Upload and retrieve URIs
workflow_orch.upload_training_data(preprocessed_data, overwrite_existing=True)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[15/Jul/25 23:12:29] MILESTONE [workflow_orchestrator._confirm_upload:453] <PROD> Ready to upload 337.9 MB of preprocessed training data across 2 date folders.
[15/Jul/25 23:12:29] INFO [workflow_orchestrator._confirm_upload:458] Target variable: cw_crypto_net_gain_crypto_inflows_winsorized
[15/Jul/25 23:12:29] INFO [workflow_orchestrator._confirm_upload:459] Target: s3://wallet-training-data/training-data-preprocessed/dda-897/[DATE]/
[15/Jul/25 23:12:32] INFO [workflow_orchestrator._confirm_upload:465] Upload cancelled


### Train Model

#### retrieve uris

In [7]:
date_suffixes = [
    '231107',
    '240306'
]

# Generate URIs for specific dates without any prior setup
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)
s3_uris

{'231107': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-897/231107/train_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-897/231107/test_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-897/231107/eval_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'val': 's3://wallet-training-data/training-data-preprocessed/dda-897/231107/val_cw_crypto_net_gain_crypto_inflows_winsorized.csv'},
 '240306': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-897/240306/train_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-897/240306/test_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-897/240306/eval_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'val': 's3://wallet-training-d

#### train model

In [9]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffix = date_suffixes[0]

modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    s3_uris,
    date_suffix
)



In [ ]:
modeling_results = modeler.train_model()

[15/Jul/25 23:12:54] INFO [wallet_modeler.train_model:82] Starting SageMaker training...
[15/Jul/25 23:12:54] INFO [image_uris._processor:530] Ignoring unnecessary instance type: None.
[15/Jul/25 23:12:54] INFO [wallet_modeler.train_model:162] Launching training job: wallet-xgb-dda-897-231107-20250715-231254
[15/Jul/25 23:12:54] INFO [wallet_modeler.train_model:163] Model output path: s3://wallet-training-data/sagemaker-models/dda-897/
[15/Jul/25 23:12:54] INFO [telemetry_logging.wrapper:91] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[15/Jul/25 23:12:54] INFO [session.submit:1053] Creating training-job with name: wallet-xgb-dda-897-231107-20250715-23

2025-07-16 06:12:55 Starting - Starting the training job...
2025-07-16 06:13:30 Downloading - Downloading input data......
2025-07-16 06:14:31 Downloading - Downloading the training image.....Arguments: train
[2025-07-16:06:15:10:INFO] Running standalone xgboost training.
[2025-07-16:06:15:10:INFO] File size need to be processed in the node: 161.58mb. Available memory size in the node: 256.37mb
[2025-07-16:06:15:10:INFO] Determined delimiter of CSV input is ','
[06:15:10] S3DistributionType set as FullyReplicated
[06:15:11] 87727x221 matrix with 19387667 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-07-16:06:15:11:INFO] Determined delimiter of CSV input is ','
[06:15:11] S3DistributionType set as FullyReplicated
[06:15:11] 5485x221 matrix with 1212185 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[06:15:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_de

#### get existing model uri

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


modeler = wm.WalletModeler(sage_wallets_config, s3_uris)
date_suffix = date_suffixes[0]


model = modeler.load_existing_model()

In [ ]:
predictions = modeler.predict_with_model()

### DDA 884 devspace